In [1]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil
import matplotlib.pyplot as plt
from PIL import Image  # 使用 PIL 來保存 RGB 格式的圖片

In [ ]:
#模型影像分割+垂直影像擺正

# 初始化 YOLO 模型
model = YOLO("C:/Users/V003479/Desktop/Project/Train/YOLOv8_TaiwanPill/models/10_30epoch_640.pt")

# 原始資料夾與輸出資料夾路徑
source_folder = "C:/Users/V003479/Desktop/Project/TaiwanPill_TEST/Taiwan_Pill_SizeDistribute_forMatting_resizeTo640"
output_folder = "C:/Users/V003479/Desktop/Project/TaiwanPill_TEST/Taiwan_Pill_Match_Database_Matting"

# 計數變數
processed_images_count = 0
# 支援的影像格式
supported_formats = {'.png', '.jpg', '.jpeg'}

# 遞迴遍歷所有影像檔案
for root, _, files in os.walk(source_folder):
    for img_file in files:
        # 確認檔案格式是否符合支援的影像格式
        if Path(img_file).suffix.lower() in supported_formats:
            img_path = os.path.join(root, img_file)
            
            # 建立對應的輸出資料夾結構
            relative_path = os.path.relpath(root, source_folder)  # 獲取相對路徑
            output_dir = os.path.join(output_folder, relative_path)  # 對應的輸出資料夾
            os.makedirs(output_dir, exist_ok=True)  # 確保對應資料夾存在
            
            # 讀取影像
            res = model.predict(source=img_path)
            img_name = Path(img_file).stem
            i = 1  # 初始化變數 i 用於文件名
            
            # 迭代每個偵測結果
            for r in res:
                img = np.copy(r.orig_img)
                
                # 迭代每個物件輪廓
                for ci, c in enumerate(r):
                    label = c.names[c.boxes.cls.tolist().pop()]
                    
                    # 創建輪廓遮罩
                    b_mask = np.zeros(img.shape[:2], np.uint8)
                    contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
                    _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
    
                    # 創建三通道遮罩
                    mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR)
                    
                    # 使用遮罩隔離物件
                    isolated = cv2.bitwise_and(mask3ch, img)
    
                    # 可選：裁切偵測物件區域
                    x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)
                    iso_crop = isolated[y1:y2, x1:x2]

                    # 如果裁切後的影像高度大於寬度，則旋轉(Y > X則旋轉)
                    if iso_crop.shape[0] > iso_crop.shape[1]:
                        iso_crop = cv2.rotate(iso_crop, cv2.ROTATE_90_CLOCKWISE)
                        
                     # 將影像轉換為 RGB 格式並使用 PIL 儲存
                    iso_crop_rgb = cv2.cvtColor(iso_crop, cv2.COLOR_BGR2RGB)
                    pil_img = Image.fromarray(iso_crop_rgb)
                    
                    # 儲存去背後的影像
                    output_file = os.path.join(output_dir, f"{img_name}_object_{i}.png")
                    pil_img.save(output_file)  # 使用 PIL 儲存為 PNG 格式
                    i += 1
    
                    # 顯示影像作為驗證
                    #plt.figure(figsize=(1, 1))
                    #plt.imshow(iso_crop_rgb)
                    #plt.axis('off')  # 隱藏座標軸
                    #plt.show()
        
            # 更新並顯示處理進度
            processed_images_count += 1
            print(f"已處理 {processed_images_count} 張原影像：{img_file}")


image 1/1 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_SizeDistribute_forMatting_resizeTo640\capsule\10_2mm\029562-20240926-002.png: 640x640 2 pill_ovals, 17.1ms
Speed: 4.7ms preprocess, 17.1ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)
已處理 1 張原影像：衛署藥製字第029562號-20240926-002.png

image 1/1 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_SizeDistribute_forMatting_resizeTo640\capsule\14mm\014145-20151230-001.png: 640x640 2 capsules, 18.9ms
Speed: 4.1ms preprocess, 18.9ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
已處理 2 張原影像：內衛藥製字第014145號-20151230-001.png

image 1/1 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_SizeDistribute_forMatting_resizeTo640\capsule\14mm\015685-20230925-001.png: 640x640 2 capsules, 16.8ms
Speed: 2.0ms preprocess, 16.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
已處理 3 張原影像：內衛藥製字第015685號-20230925-001.png

image 1/1 C:\Users\V003479\Desktop\Project\TaiwanPill

In [1]:
import os
import cv2
import numpy as np
import time  # 用來生成唯一的時間戳

def extract_segmented_images(image_dir, label_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    print(f"共找到 {total_images} 張圖片，開始處理...")

    for idx, image_file in enumerate(image_files, start=1):
        print(f"[{idx}/{total_images}] 處理圖片: {image_file}")
        main_name = image_file.split('-')[1] if '-' in image_file else os.path.splitext(image_file)[0]
        label_file = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')

        if os.path.exists(label_file):
            image_path = os.path.join(image_dir, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            if image is None:
                print(f"  - 無法讀取圖片: {image_path}，跳過此圖片。")
                continue

            h, w, _ = image.shape
            with open(label_file, 'r') as f:
                lines = f.readlines()

            if not lines:
                print(f"  - 標籤檔案為空: {label_file}")
                continue

            count = 1
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 5:
                    class_id = parts[0]
                    polygon = np.array([float(coord) for coord in parts[1:]], dtype=np.float32).reshape(-1, 2)

                    # 轉換比例座標到像素座標
                    polygon[:, 0] *= w
                    polygon[:, 1] *= h
                    polygon = polygon.astype(np.int32)

                    # 建立遮罩
                    mask = np.zeros((h, w), dtype=np.uint8)
                    cv2.fillPoly(mask, [polygon], 255)

                    # 添加 alpha 通道，將遮罩部分設置為不透明，其他部分設置為透明
                    b, g, r = cv2.split(image)  # 分離 RGB 通道
                    alpha = mask  # 遮罩為 alpha 通道
                    transparent_image = cv2.merge([b, g, r, alpha])  # 合併 RGBA 圖像

                    # 計算多邊形範圍並裁剪
                    x, y, rect_w, rect_h = cv2.boundingRect(polygon)
                    cropped_image = transparent_image[y:y + rect_h, x:x + rect_w]

                    # 確認裁剪結果
                    if cropped_image.size == 0:
                        print(f"  - 裁剪結果為空，跳過此區域。")
                        continue

                    # 如果裁切後的影像高度大於寬度，則旋轉
                    if cropped_image.shape[0] > cropped_image.shape[1]:
                        cropped_image = cv2.rotate(cropped_image, cv2.ROTATE_90_CLOCKWISE)
                        print(f"  - 影像已旋轉")

                    # 為避免同名檔案覆蓋，加入時間戳或編號
                    timestamp = int(time.time() * 1000)  # 毫秒級別的時間戳
                    output_name = f"{main_name}_{count}_{timestamp}.png"
                    output_path = os.path.join(output_dir, output_name)

                    # 儲存帶透明背景的裁剪圖片
                    cv2.imwrite(output_path, cropped_image)
                    print(f"  - 已儲存分割圖片: {output_name}")
                    count += 1
        else:
            print(f"  - 未找到標籤檔案: {label_file}")
    
    print("處理完成！")

if __name__ == "__main__":
    # 設定路徑
    image_dir = "C:/Users/V003479/Desktop/Project/TaiwanPill_TEST/Taiwan_Pill_SizeDistribute_Labeled_Original/train/images"  # 替換為您的圖片資料夾路徑
    label_dir = "C:/Users/V003479/Desktop/Project/TaiwanPill_TEST/Taiwan_Pill_SizeDistribute_Labeled_Original/train/labels"  # 替換為您的標籤資料夾路徑
    output_dir = "C:/Users/V003479/Desktop/Project/TaiwanPill_TEST/Taiwan_Pill_Match_Segmentaion_MattingbyLabel_transparency"  # 替換為輸出的資料夾路徑

    extract_segmented_images(image_dir, label_dir, output_dir)

共找到 2001 張圖片，開始處理...
[1/2001] 處理圖片: -000006-20230922-001_jpg.rf.af2d14c487559c404700ecee58215ce3.jpg
  - 已儲存分割圖片: 000006_1_1734425629535.png
  - 已儲存分割圖片: 000006_2_1734425629544.png
[2/2001] 處理圖片: -000006-20230922-002_jpg.rf.093590d4fba381f76caae9b63c4b1df4.jpg
  - 已儲存分割圖片: 000006_1_1734425629552.png
  - 已儲存分割圖片: 000006_2_1734425629555.png
[3/2001] 處理圖片: -000007-20240329-001_jpg.rf.dc8e01a694a7facf199ad8a73ed097cb.jpg
  - 已儲存分割圖片: 000007_1_1734425629645.png
  - 已儲存分割圖片: 000007_2_1734425629677.png
[4/2001] 處理圖片: -000010-20231103-001_jpeg.rf.1ee077fd4b508d413882121e112c532c.jpg
  - 影像已旋轉
  - 已儲存分割圖片: 000010_1_1734425629841.png
  - 影像已旋轉
  - 已儲存分割圖片: 000010_2_1734425629908.png
[5/2001] 處理圖片: -000014-20230627-001_png.rf.014a40f2257c503c6a86cee99c895608.jpg
  - 已儲存分割圖片: 000014_1_1734425629956.png
  - 已儲存分割圖片: 000014_2_1734425629972.png
[6/2001] 處理圖片: -000019-20230628-001_png.rf.78a44b67cdba0ac7580105e1ce7f8be4.jpg
  - 已儲存分割圖片: 000019_1_1734425630005.png
  - 已儲存分割圖片: 000019_2_1734425630017.pn

In [2]:
#將檔名對應之後放入分類資料夾

import os
import shutil

def get_image_info(image_path):
    """
    根據圖片路徑提取名稱和分類信息。
    提取出圖片的名稱和所在的分類及尺寸。
    """
    # 取得檔案名稱
    filename = os.path.basename(image_path)
    
    # 假設檔名中間的數字（例如057107）為名稱
    name = filename.split('-')[0].split('第')[-1].split('號')[0]  # 提取名稱（假設固定格式）

    # 假設路徑中的資料夾層級結構
    category_path = os.path.dirname(image_path)
    parts = category_path.split(os.sep)  # 依照路徑分割

    # 檢查是否是 capsule 或 pill
    if 'capsule' in parts:
        category = 'capsule'
        size = parts[parts.index('capsule') + 1]  # 獲取尺寸（例如 8mm）
        return name, category, size
    elif 'pill' in parts:
        category = 'pill'
        shape = parts[parts.index('pill') + 1]  # 獲取形狀（例如 doubleround）
        size = parts[parts.index('pill') + 2]   # 獲取尺寸（例如 9mm）
        return name, category, shape, size
    else:
        raise ValueError(f"未知的分類類型: {category_path}")  # 輸出具體的錯誤資訊

def organize_images(source_folder, target_folder, reference_folder):
    """
    根據來源資料夾中的圖片名稱與 reference_folder 資料夾中的圖片名稱對應，
    並根據提取的分類和尺寸創建新資料夾並移動圖片。
    """
    # 遍歷 source_folder 中的圖片
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):  # 根據需要的檔案格式調整
                source_image_path = os.path.join(root, file)

                # 提取來源圖片的名稱、分類、形狀（如果有）和尺寸
                try:
                    info = get_image_info(source_image_path)
                except ValueError as e:
                    print(f"Error processing {source_image_path}: {e}")
                    continue  # 如果遇到未知的分類類型，跳過該檔案

                name = info[0]
                category = info[1]

                if category == 'capsule':
                    size = info[2]
                    # 在目標資料夾中創建capsule類型的尺寸資料夾
                    target_category_folder = os.path.join(target_folder, category, size)
                elif category == 'pill':
                    shape = info[2]
                    size = info[3]
                    # 在目標資料夾中創建pill類型的形狀和尺寸資料夾
                    target_category_folder = os.path.join(target_folder, category, shape, size)
                
                os.makedirs(target_category_folder, exist_ok=True)

                # 檢查 reference_folder 中是否有對應名稱的圖片
                reference_image_base_name = f"{name}"  # 假設來源檔案名稱為 057107，不帶後綴
                matched_files = [file for file in os.listdir(reference_folder) if file.startswith(reference_image_base_name)]
                
                if matched_files:
                    # 如果找到匹配的圖片，將它們都移動到新資料夾
                    for ref_file in matched_files:
                        reference_image_path = os.path.join(reference_folder, ref_file)

                        # 創建目標路徑，並將檔案移動到新資料夾中
                        target_image_path = os.path.join(target_category_folder, ref_file)
                        shutil.copy(reference_image_path, target_image_path)
                        print(f"移動圖片 {ref_file} 到 {target_image_path}")
                else:
                    print(f"未找到對應圖片: {reference_image_base_name}")

if __name__ == "__main__":
    source_folder = r'C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_SizeDistribute_forMatting_resizeTo640'  # 請根據實際路徑調整
    reference_folder = r'C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Segmentaion_MattingbyLabel_transparency'  # 請根據實際路徑調整
    target_folder = r'C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel_Transparency'  # 請根據實際路徑調整

    organize_images(source_folder, target_folder, reference_folder)

移動圖片 029562_1_1734425692141.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel__Transparency\capsule\10_2mm\029562_1_1734425692141.png
移動圖片 029562_2_1734425692180.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel__Transparency\capsule\10_2mm\029562_2_1734425692180.png
移動圖片 014145_1_1734425640848.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel__Transparency\capsule\14mm\014145_1_1734425640848.png
移動圖片 014145_2_1734425640853.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel__Transparency\capsule\14mm\014145_2_1734425640853.png
移動圖片 015685_1_1734425642371.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_Database_MattingbyLabel__Transparency\capsule\14mm\015685_1_1734425642371.png
移動圖片 015685_2_1734425642387.png 到 C:\Users\V003479\Desktop\Project\TaiwanPill_TEST\Taiwan_Pill_Match_